# Use pyquery-ql.py

Send a graphql query to GitHub
and pretty print output.

Supports Python 3.6+

In [1]:
import json
import os
import pprint

import requests

In [2]:
# get api token and set authorization
api_token = os.environ['GITHUB_API_TOKEN']
headers = {'Authorization': f'token {api_token}'}

In [3]:
# set url to a graphql endpoint
url = 'https://api.github.com/graphql'

In [4]:
# add a json query
query = """
{
  organization(login: "jupyterhub") {
    repositories(first: 30) {
      totalCount
      edges {
        node {
          name
          url
          issues(states: OPEN) {
            totalCount
          }
          pullRequests(states: OPEN) {
            totalCount
          }
        }
      }
    }
  }
}
"""

In [5]:
# submit the request
r = requests.post(url=url, json={'query': query}, headers=headers)

In [22]:
data = json.loads(r.text)
data

{'data': {'organization': {'repositories': {'edges': [{'node': {'issues': {'totalCount': 143},
       'name': 'jupyterhub',
       'pullRequests': {'totalCount': 6},
       'url': 'https://github.com/jupyterhub/jupyterhub'}},
     {'node': {'issues': {'totalCount': 10},
       'name': 'configurable-http-proxy',
       'pullRequests': {'totalCount': 0},
       'url': 'https://github.com/jupyterhub/configurable-http-proxy'}},
     {'node': {'issues': {'totalCount': 13},
       'name': 'oauthenticator',
       'pullRequests': {'totalCount': 2},
       'url': 'https://github.com/jupyterhub/oauthenticator'}},
     {'node': {'issues': {'totalCount': 22},
       'name': 'dockerspawner',
       'pullRequests': {'totalCount': 1},
       'url': 'https://github.com/jupyterhub/dockerspawner'}},
     {'node': {'issues': {'totalCount': 2},
       'name': 'sudospawner',
       'pullRequests': {'totalCount': 0},
       'url': 'https://github.com/jupyterhub/sudospawner'}},
     {'node': {'issues': {'to

In [ ]:
# pprint.pprint(data)

Walk the JSON response contents

In [12]:
def traverse(obj, path=None, callback=None):
    if path is None:
        path = []

    if isinstance(obj, dict):
        value = {k: traverse(v, path + [k], callback)
                 for k, v in obj.items()}
    elif isinstance(obj, list):
        value = [traverse(elem, path + [[]], callback)
                 for elem in obj]
    else:
        value = obj

    if callback is None:  # if a callback is provided, call it to get the new value
        return value
    else:
        return callback(path, value)

In [47]:
traverse(data)

{'data': {'organization': {'repositories': {'edges': [{'node': {'issues': {'totalCount': 143},
       'name': 'jupyterhub',
       'pullRequests': {'totalCount': 6},
       'url': 'https://github.com/jupyterhub/jupyterhub'}},
     {'node': {'issues': {'totalCount': 10},
       'name': 'configurable-http-proxy',
       'pullRequests': {'totalCount': 0},
       'url': 'https://github.com/jupyterhub/configurable-http-proxy'}},
     {'node': {'issues': {'totalCount': 13},
       'name': 'oauthenticator',
       'pullRequests': {'totalCount': 2},
       'url': 'https://github.com/jupyterhub/oauthenticator'}},
     {'node': {'issues': {'totalCount': 22},
       'name': 'dockerspawner',
       'pullRequests': {'totalCount': 1},
       'url': 'https://github.com/jupyterhub/dockerspawner'}},
     {'node': {'issues': {'totalCount': 2},
       'name': 'sudospawner',
       'pullRequests': {'totalCount': 0},
       'url': 'https://github.com/jupyterhub/sudospawner'}},
     {'node': {'issues': {'to

In [15]:
def traverse_modify(obj, target_path, action):
    target_path = to_path(target_path)  # converts 'foo.bar' to ['foo', 'bar']

    # This will get called for every path/value in the structure
    def transformer(path, value):
        if path == target_path:
            return action(value)
        else:
            return value

    return traverse(obj, callback=transformer)

In [16]:
def to_path(path):
    """
    Helper function, converting path strings into path lists.
        >>> to_path('foo')
        ['foo']
        >>> to_path('foo.bar')
        ['foo', 'bar']
        >>> to_path('foo.bar[]')
        ['foo', 'bar', []]
    """
    if isinstance(path, list):
        return path  # already in list format

    def _iter_path(path):
        for parts in path.split('[]'):
            for part in parts.strip('.').split('.'):
                yield part
            yield []

    return list(_iter_path(path))[:-1]

In [64]:
from operator import itemgetter

def sort_nodes(names):
    """Will sort a list of points."""
    return sorted(names, reverse=True, key=itemgetter('node.name'))

traverse_modify(data, 'data.organization.repositories.edges[].node{}', sort_nodes)

{'data': {'organization': {'repositories': {'edges': [{'node': {'issues': {'totalCount': 143},
       'name': 'jupyterhub',
       'pullRequests': {'totalCount': 6},
       'url': 'https://github.com/jupyterhub/jupyterhub'}},
     {'node': {'issues': {'totalCount': 10},
       'name': 'configurable-http-proxy',
       'pullRequests': {'totalCount': 0},
       'url': 'https://github.com/jupyterhub/configurable-http-proxy'}},
     {'node': {'issues': {'totalCount': 13},
       'name': 'oauthenticator',
       'pullRequests': {'totalCount': 2},
       'url': 'https://github.com/jupyterhub/oauthenticator'}},
     {'node': {'issues': {'totalCount': 22},
       'name': 'dockerspawner',
       'pullRequests': {'totalCount': 1},
       'url': 'https://github.com/jupyterhub/dockerspawner'}},
     {'node': {'issues': {'totalCount': 2},
       'name': 'sudospawner',
       'pullRequests': {'totalCount': 0},
       'url': 'https://github.com/jupyterhub/sudospawner'}},
     {'node': {'issues': {'to

In [ ]:
issue_data = data['data']

In [ ]:
org = issue_data['organization']

In [ ]:
repos = org['repositories']

In [ ]:
edges = repos['edges']

In [ ]:
# edges[0]['node']

In [ ]:
# print(edges[0]['node']['name'] + '---' + edges[0]['node']['url'])

In [ ]:
# for edge in edges:
#    pprint.pprint(edge)

Bring into pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_records(edges)

In [ ]:
df.columns

In [ ]:
# df['node']

Generate basic report of total open issues

In [ ]:
print(f"{'Repo':30} {'OpenIssues':11} {'OpenPRs':9} {'URL'}")

line = 26
while line > 0:
    lineout = f"{df['node'][line]['name']:30} {(df['node'][line]['issues']['totalCount']):8} {(df['node'][line]['pullRequests']['totalCount']):8}     {df['node'][line]['url']}"
    print(lineout)
    line -= 1

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

In [ ]:
df.sort_index

In [ ]:
# output data to a csv
# df.to_csv('issue_data.csv')